In [86]:
import numpy as np
import math
from scipy.stats import multivariate_normal
class igmm:
    def __init__(self, X, dim, sigma_ini, tau):
        self.X = X
        self.dim = dim
        self.pi = list()
        self.mu = list()
        self.C = list()
        self.sp = list()
        self.sigma_ini = sigma_ini
        self.tau = tau
        
    def posterior_prob(self, x, component):
        mvn = multivariate_normal(mean = np.array(self.mu[component][0]), cov = np.array(self.C[component]))
        pdf_value = mvn.pdf(x)
        return self.pi[component]*pdf_value
    
    def createnewornot(self, x):
        createnew = True
        for i in range(len(self.pi)):
            novelty_criterion = self.tau/(((2*math.pi)**(dim/2))*math.sqrt(np.linalg.det(np.array(self.C[i]))))
            #print(f"Is this not a vector of length 2: {self.mu[i]}\n Dimensions: {np.shape(self.mu[i])}")
            mvn = multivariate_normal(mean = self.mu[i][0], cov = np.array(self.C[i]))
            pdf_value = mvn.pdf(x)
            if(pdf_value >= novelty_criterion):
                createnew = False
                return createnew
        return createnew
    
    def update(self, x):
        createnew = self.createnewornot(x)
        if(createnew == True):
            self.mu.append(np.array(x))
            self.C.append((self.sigma_ini**2)*np.eye(dim))
            self.sp.append(1)
            
            total_sum = np.sum(self.sp)
            for j in range(len(self.sp)-1):
                self.pi[j] = self.sp[j]/total_sum
            self.pi.append(self.sp[len(self.sp)-1]/total_sum)
        else:
            for j in range(len(self.sp)):
                posterior_value = self.posterior_prob(x,j)
                self.sp[j] += posterior_value
                prev_mu = self.mu[j]
                self.mu[j] = self.mu[j] + (posterior_value/self.sp[j])*(np.array(x)-self.mu[j])
                self.C[j] = self.C[j] - np.matmul(self.mu[j]-prev_mu,(self.mu[j]-prev_mu).T) + (posterior_value/self.sp[j])*(np.matmul((np.array(x)-self.mu[j]),(np.array(x)-self.mu[j]).T)-self.C[j])
            total_sum = np.sum(self.sp)
            for j in range(len(self.pi)):
                self.pi[j] = self.sp[j]/total_sum
        return
    
    def fit(self):
        iter = 1
        for x in self.X:
            self.update(x)
            print(f"\n\n\033[1mFor next sample {iter}:\033[0m")
            print_pi = np.array(self.pi)
            print_mean = np.array(self.mu)
            print_cov = np.array(self.C)
            print_sp = np.array(self.sp)
            for j in range(len(self.mu)):
                print(f"\033[1mComponent #{j+1}:\033[0m\nPrior Probability: {print_pi[j]}\nMean: {print_mean[j][0]}\nContribution of dataset: {print_sp[j]}\nCovariance:\n {print_cov[j]}")
            iter+=1
        return

In [44]:
class gen_samples:
    def __init__(self, pi, mu, C):
        self.X = list()
        self.pi = pi
        self.mu = mu
        self.C = C
        return
    
    def generate_samples(self, n_samples):
        iter = 1
        while(iter <= n_samples):
            z_i = np.argmax(np.random.multinomial(1, self.pi))
            sample = np.random.multivariate_normal(self.mu[z_i], self.C[z_i], 1)
            self.X.append(np.array(sample))
            iter += 1
        return

In [78]:
pi_value = [1/4, 3/4]
pi_value = np.array(pi_value)
mu_value = [[-5,0,0], [10,0,5]]
mu_value = np.array(mu_value)
C_value = [[[1,0,0],[0,1,0],[0,0,1]],[[1,0,0],[0,1,0],[0,0,1]]]
C_value = np.array(C_value)
sample_generator = gen_samples(pi_value, mu_value, C_value)
sample_generator.generate_samples(1000)
X = sample_generator.X
X = np.array(X)

In [87]:
sigma_threshold = (np.max(X)-np.min(X))/10
tau = 0.01
dim = 3
incremental_model = igmm(X,dim,sigma_threshold,tau)
incremental_model.fit()



For next sample 1:
Component #1:
Prior Probability: 1.0
Mean: [10.4431625   1.27670067  4.66744712]
Contribution of dataset: 1
Covariance:
 [[4.25530779 0.         0.        ]
 [0.         4.25530779 0.        ]
 [0.         0.         4.25530779]]


For next sample 2:
Component #1:
Prior Probability: 1.0
Mean: [10.44420579  1.26775826  4.66718452]
Contribution of dataset: 1.0044661182585095
Covariance:
 [[4.25439006 0.01800248 0.01800248]
 [0.01800248 4.25439006 0.01800248]
 [0.01800248 0.01800248 4.25439006]]


For next sample 3:
Component #1:
Prior Probability: 1.0
Mean: [10.43781807  1.26402843  4.66871077]
Contribution of dataset: 1.0070192418341086
Covariance:
 [[4.26593245 0.0402855  0.0402855 ]
 [0.0402855  4.26593245 0.0402855 ]
 [0.0402855  0.0402855  4.26593245]]


For next sample 4:
Component #1:
Prior Probability: 1.0
Mean: [10.43938439  1.25755094  4.67256485]
Contribution of dataset: 1.0097156618879175
Covariance:
 [[4.27655545 0.06219299 0.06219299]
 [0.06219299 4.276



For next sample 232:
Component #1:
Prior Probability: 0.5563883318440062
Mean: [10.31744909  0.91226064  4.73310418]
Contribution of dataset: 1.4438907739600386
Covariance:
 [[4.02983435 1.08272238 1.08272238]
 [1.08272238 4.02983435 1.08272238]
 [1.08272238 1.08272238 4.02983435]]
Component #2:
Prior Probability: 0.44361166815599395
Mean: [-5.19662339  0.195425   -0.28859998]
Contribution of dataset: 1.1512225512504917
Covariance:
 [[3.98856746 0.29222935 0.29222935]
 [0.29222935 3.98856746 0.29222935]
 [0.29222935 0.29222935 3.98856746]]


For next sample 233:
Component #1:
Prior Probability: 0.5556729163040631
Mean: [10.31744909  0.91226064  4.73310418]
Contribution of dataset: 1.4438907739600388
Covariance:
 [[4.02983435 1.08272238 1.08272238]
 [1.08272238 4.02983435 1.08272238]
 [1.08272238 1.08272238 4.02983435]]
Component #2:
Prior Probability: 0.4443270836959369
Mean: [-5.19656861  0.19355187 -0.28785909]
Contribution of dataset: 1.154563697356941
Covariance:
 [[3.97841609 0.

Component #1:
Prior Probability: 0.6056163477806754
Mean: [10.24990728  0.70920422  4.75394601]
Contribution of dataset: 1.9937944249996316
Covariance:
 [[3.66548498 1.53120888 1.53120888]
 [1.53120888 3.66548498 1.53120888]
 [1.53120888 1.53120888 3.66548498]]
Component #2:
Prior Probability: 0.3943836522193247
Mean: [-5.17807985  0.19041016 -0.2501215 ]
Contribution of dataset: 1.298379625958594
Covariance:
 [[3.81377675 0.53637799 0.53637799]
 [0.53637799 3.81377675 0.53637799]
 [0.53637799 0.53637799 3.81377675]]


For next sample 446:
Component #1:
Prior Probability: 0.6060329712099234
Mean: [10.24702929  0.70932717  4.75507086]
Contribution of dataset: 1.9972759266039846
Covariance:
 [[3.66455318 1.53399739 1.53399739]
 [1.53399739 3.66455318 1.53399739]
 [1.53399739 1.53399739 3.66455318]]
Component #2:
Prior Probability: 0.39396702879007667
Mean: [-5.17807985  0.19041016 -0.2501215 ]
Contribution of dataset: 1.2983796259585996
Covariance:
 [[3.81377675 0.53637799 0.53637799]
 [



For next sample 696:
Component #1:
Prior Probability: 0.6760185655451235
Mean: [10.17552251  0.52398838  4.8020442 ]
Contribution of dataset: 2.971589407857858
Covariance:
 [[3.20700475 1.77500752 1.77500752]
 [1.77500752 3.20700475 1.77500752]
 [1.77500752 1.77500752 3.20700475]]
Component #2:
Prior Probability: 0.3239814344548764
Mean: [-5.15759014  0.16710743 -0.2403321 ]
Contribution of dataset: 1.4241321881335873
Covariance:
 [[3.68049519 0.69249462 0.69249462]
 [0.69249462 3.68049519 0.69249462]
 [0.69249462 0.69249462 3.68049519]]


For next sample 697:
Component #1:
Prior Probability: 0.6761464155582085
Mean: [10.17545982  0.52345465  4.80334367]
Contribution of dataset: 2.9733247385460584
Covariance:
 [[3.20851518 1.77735371 1.77735371]
 [1.77735371 3.20851518 1.77735371]
 [1.77735371 1.77735371 3.20851518]]
Component #2:
Prior Probability: 0.3238535844417914
Mean: [-5.15759014  0.16710743 -0.2403321 ]
Contribution of dataset: 1.4241321881335873
Covariance:
 [[3.68049519 0.6

Component #1:
Prior Probability: 0.6338203888500001
Mean: [10.11831351  0.39009356  4.83836166]
Contribution of dataset: 4.486016886489543
Covariance:
 [[2.79082239 1.84225089 1.84225089]
 [1.84225089 2.79082239 1.84225089]
 [1.84225089 1.84225089 2.79082239]]
Component #2:
Prior Probability: 0.22473265896760633
Mean: [-5.13387415  0.14745651 -0.21434201]
Contribution of dataset: 1.5905996727299465
Covariance:
 [[3.56739068 0.89210547 0.89210547]
 [0.89210547 3.56739068 0.89210547]
 [0.89210547 0.89210547 3.56739068]]
Component #3:
Prior Probability: 0.14144695218239345
Mean: [11.00549319 -2.83365239  4.39584626]
Contribution of dataset: 1.001124967254508
Covariance:
 [[4.26382567 0.01329959 0.01329959]
 [0.01329959 4.26382567 0.01329959]
 [0.01329959 0.01329959 4.26382567]]
